# Battery Remaining Useful Life (RUL) Prediction
This notebook demonstrates the process of predicting the Remaining Useful Life (RUL) of batteries using the K-Nearest Neighbors (KNN) algorithm. The dataset contains various features related to battery usage cycles and characteristics.

In [1]:
import os

# Install git if not already installed
!apt-get install -y git

# Clone the repository
!git clone https://github.com/Elite-AI-Club/AI-Driven-Innovation-Electronics.git

# Change working directory to the repository
%cd AI-Driven-Innovation-Electronics/6_Battery_RUL

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Cloning into 'AI-Driven-Innovation-Electronics'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (78/78), done.
remote: Total 100 (delta 29), reused 85 (delta 17), pack-reused 0 (from 0)
Receiving objects: 100% (100/100), 20.64 MiB | 23.07 MiB/s, done.
Resolving deltas: 100% (29/29), done.
/content/AI-Driven-Innovation-Electronics/6_Battery_RUL


In [17]:
import pandas as pd
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns


## Load and Prepare the Data
First, load the dataset and prepare it for the model. This involves selecting relevant features and the target variable.

In [18]:
pwd

'/content/AI-Driven-Innovation-Electronics/6_Battery_RUL'

In [5]:
data = pd.read_csv('Battery_RUL.csv')
data

,Cycle_Index,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s),RUL
0,1.0,2595.30,1151.488500,3.670,3.211,5460.001,6755.01,10777.82,1112
1,2.0,7408.64,1172.512500,4.246,3.220,5508.992,6762.02,10500.35,1111
2,3.0,7393.76,1112.992000,4.249,3.224,5508.993,6762.02,10420.38,1110
3,4.0,7385.50,1080.320667,4.250,3.225,5502.016,6762.02,10322.81,1109
4,6.0,65022.75,29813.487000,4.290,3.398,5480.992,53213.54,56699.65,1107
...,...,...,...,...,...,...,...,...,...
15059,1108.0,770.44,179.523810,3.773,3.742,922.775,1412.38,6678.88,4
15060,1109.0,771.12,179.523810,3.773,3.744,915.512,1412.31,6670.38,3
15061,1110.0,769.12,179.357143,3.773,3.742,915.513,1412.31,6637.12,2
15062,1111.0,773.88,162.374667,3.763,3.839,539.375,1148.00,7660.62,1


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15064 entries, 0 to 15063
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Cycle_Index                15064 non-null  float64
 1   Discharge Time (s)         15064 non-null  float64
 2   Decrement 3.6-3.4V (s)     15064 non-null  float64
 3   Max. Voltage Dischar. (V)  15064 non-null  float64
 4   Min. Voltage Charg. (V)    15064 non-null  float64
 5   Time at 4.15V (s)          15064 non-null  float64
 6   Time constant current (s)  15064 non-null  float64
 7   Charging time (s)          15064 non-null  float64
 8   RUL                        15064 non-null  int64  
dtypes: float64(8), int64(1)
memory usage: 1.0 MB


In [7]:
data.describe()

,Cycle_Index,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s),RUL
count,15064.000000,15064.000000,15064.000000,15064.000000,15064.000000,15064.000000,15064.000000,15064.000000,15064.000000
mean,556.155005,4581.273960,1239.784672,3.908176,3.577904,3768.336171,5461.266970,10066.496204,554.194172
std,322.378480,33144.012077,15039.589269,0.091003,0.123695,9129.552477,25155.845202,26415.354121,322.434514
min,1.000000,8.690000,-397645.908000,3.043000,3.022000,-113.584000,5.980000,5.980000,0.000000
25%,271.000000,1169.310000,319.600000,3.846000,3.488000,1828.884179,2564.310000,7841.922500,277.000000
50%,560.000000,1557.250000,439.239471,3.906000,3.574000,2930.203500,3824.260000,8320.415000,551.000000
75%,833.000000,1908.000000,600.000000,3.972000,3.663000,4088.326500,5012.350000,8763.282500,839.000000
max,1134.000000,958320.370000,406703.768000,4.363000,4.379000,245101.117000,880728.100000,880728.100000,1133.000000


## Extract the features and the target variable


In [8]:
X = data.iloc[:, 1:-1]
y = data['RUL']

In [9]:
X

,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s)
0,2595.30,1151.488500,3.670,3.211,5460.001,6755.01,10777.82
1,7408.64,1172.512500,4.246,3.220,5508.992,6762.02,10500.35
2,7393.76,1112.992000,4.249,3.224,5508.993,6762.02,10420.38
3,7385.50,1080.320667,4.250,3.225,5502.016,6762.02,10322.81
4,65022.75,29813.487000,4.290,3.398,5480.992,53213.54,56699.65
...,...,...,...,...,...,...,...
15059,770.44,179.523810,3.773,3.742,922.775,1412.38,6678.88
15060,771.12,179.523810,3.773,3.744,915.512,1412.31,6670.38
15061,769.12,179.357143,3.773,3.742,915.513,1412.31,6637.12
15062,773.88,162.374667,3.763,3.839,539.375,1148.00,7660.62


In [10]:
y

,RUL
0,1112
1,1111
2,1110
3,1109
4,1107
...,...
15059,4
15060,3
15061,2
15062,1


## Split the Data
Split the data into training and test sets to evaluate the model's performance effectively.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Standardize the Features
Standardizing the features is crucial for many machine learning models to perform well, including KNN.

In [12]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Train the KNN Model
Train the KNN model using the scaled training data.

In [13]:
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

KNeighborsRegressor()

## Evaluate the Model
Evaluate the model using Root Mean Squared Error (RMSE) on the test set.

In [14]:
y_pred = knn.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f'Root Mean Squared Error: {rmse}')

Root Mean Squared Error: 37.60927373613025


In [15]:
# train accuracy R sqared
print(f'Train Accuracy: {knn.score(X_train_scaled, y_train)}')

Train Accuracy: 0.9933708187909873


In [16]:
# performance on test data
print(f'Test Accuracy: {knn.score(X_test_scaled, y_test)}')

Test Accuracy: 0.9863450390972913
